In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D,Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

C:\Users\prash\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
os.getcwd()

'E:\\Insofe\\Python Lab'

Link for the dataset:
https://archive.ics.uci.edu/ml/machine-learning-databases/20newsgroups-mld/

In [3]:
import os
#GLOVE_DIR = 'glove.6B/'
glove_data = 'glove.6B.100d.txt'
TEXT_DATA_DIR = '20news_18828/'

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
nb_epochs = 50

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(glove_data,encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')


Indexing word vectors.
Found 400000 word vectors.
Processing text dataset


In [9]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
#                 f = open(fpath)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 18828 texts.


In [10]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Found 172668 unique tokens.
Shape of data tensor: (18828, 1000)
Shape of label tensor: (18828, 20)


In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels)

In [13]:
TEXT_DATA_DIR = '20news_18828/'
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

Indexing word vectors.


In [14]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

## LSTM - GLOVE Vectors

In [18]:
print('Training model.')

from keras.layers import LSTM
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
z = Dropout(0.2)(embedded_sequences)
z = LSTM(128)(z)
z = Dropout(0.4)(z)
preds_lstm = Dense(20, activation='softmax')(z)


from keras.optimizers import Adam
adam = Adam(lr=0.001)
model_lstm = Model(sequence_input, preds_lstm)
model_lstm.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])

model_lstm_hist = model_lstm.fit(x_train, y_train,
                  batch_size=128,
                  epochs=nb_epochs,
                  validation_data=(x_test, y_test)).history

Training model.
Train on 14121 samples, validate on 4707 samples
Epoch 1/50
14121/14121 [==============================] - 411s 29ms/step - loss: 2.7323 - acc: 0.1333 - val_loss: 2.3436 - val_acc: 0.2384
Epoch 2/50
14121/14121 [==============================] - 439s 31ms/step - loss: 2.2328 - acc: 0.2403 - val_loss: 2.0433 - val_acc: 0.3023
Epoch 3/50
14121/14121 [==============================] - 403s 29ms/step - loss: 2.1251 - acc: 0.2843 - val_loss: 1.9540 - val_acc: 0.3535
Epoch 4/50
14121/14121 [==============================] - 416s 29ms/step - loss: 1.8648 - acc: 0.3644 - val_loss: 1.6169 - val_acc: 0.4631
Epoch 5/50
14121/14121 [==============================] - 405s 29ms/step - loss: 1.6480 - acc: 0.4494 - val_loss: 1.4897 - val_acc: 0.4929
Epoch 6/50
14121/14121 [==============================] - 431s 31ms/step - loss: 1.4857 - acc: 0.4952 - val_loss: 1.2859 - val_acc: 0.5681
Epoch 7/50
14121/14121 [==============================] - 349s 25ms/step - loss: 1.3142 - acc: 0.5534

## 1D-CNN - GLOVE Vectors

In [30]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

from keras.optimizers import Adam
adam = Adam(lr=0.001)
model1 = Model(sequence_input, preds)
model1.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])

model1_hist = model1.fit(x_train, y_train,
              batch_size=128,
              epochs=nb_epochs,
              validation_data=(x_val, y_val)).history

Training model.
Train on 15063 samples, validate on 3765 samples
Epoch 1/50
15063/15063 [==============================] - 2s - loss: 2.3406 - acc: 0.2369 - val_loss: 1.8085 - val_acc: 0.3880
Epoch 2/50
15063/15063 [==============================] - 2s - loss: 1.3878 - acc: 0.5302 - val_loss: 1.1940 - val_acc: 0.5997
Epoch 3/50
15063/15063 [==============================] - 2s - loss: 1.1086 - acc: 0.6241 - val_loss: 1.1211 - val_acc: 0.6218
Epoch 4/50
15063/15063 [==============================] - 2s - loss: 0.9185 - acc: 0.6930 - val_loss: 0.9874 - val_acc: 0.6637
Epoch 5/50
15063/15063 [==============================] - 2s - loss: 0.7551 - acc: 0.7489 - val_loss: 0.8764 - val_acc: 0.7150
Epoch 6/50
15063/15063 [==============================] - 2s - loss: 0.6309 - acc: 0.7908 - val_loss: 0.8812 - val_acc: 0.7102
Epoch 7/50
15063/15063 [==============================] - 2s - loss: 0.5078 - acc: 0.8334 - val_loss: 0.8623 - val_acc: 0.7325
Epoch 8/50
15063/15063 [======================